In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip install -r requirements.txt

Cloning into 'yolov9'...
remote: Enumerating objects: 781, done.
remote: Total 781 (delta 0), reused 0 (delta 0), pack-reused 781 (from 1)
Receiving objects: 100% (781/781), 3.27 MiB | 21.60 MiB/s, done.
Resolving deltas: 100% (331/331), done.
/content/yolov9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.5 MB/s eta 0:00:00


In [ ]:
!cp "/content/drive/MyDrive/Models/weights-b/best-b.pt" .
!cp "/content/drive/MyDrive/Models/weights-b/last-b.pt" .
!cp "/content/drive/MyDrive/Models/weights-s/best-s.pt" .
!cp "/content/drive/MyDrive/Models/weights-s/last-s.pt" .

In [ ]:
!python export.py --weights best-b.pt --img-size 640 --include onnx --dynamic --simplify --opset 12
!python export.py --weights last-b.pt --img-size 640 --include onnx --dynamic --simplify --opset 12
!python export.py --weights best-s.pt --img-size 640 --include onnx --dynamic --simplify --opset 12
!python export.py --weights last-s.pt --img-size 640 --include onnx --dynamic --simplify --opset 12

export: data=data/coco.yaml, weights=['best-b.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=True, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
yolov9-c summary: 604 layers, 50702898 parameters, 0 gradients, 236.6 GFLOPs

PyTorch: starting from best-b.pt with output shape (1, 7, 8400) (98.0 MB)
requirements: YOLO requirement "onnx" not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.6 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0...
requirements: YOLO requirements "onnxruntime" "onnx-simplifier>=0.4.1" not found, attempting AutoUpdate...

In [ ]:
import onnx

In [ ]:
onnx_files = [
    "/content/yolov9/best-b.onnx",
    "/content/yolov9/last-b.onnx",
    "/content/yolov9/best-s.onnx",
    "/content/yolov9/last-s.onnx"
]

In [ ]:
for onnx_file in onnx_files:
    print(f"Checking {onnx_file}...")
    onnx_model = onnx.load(onnx_file)
    onnx.checker.check_model(onnx_model)
    print(f"{onnx_file} is valid!")
    print(f"Inputs for {onnx_file}:")
    for input in onnx_model.graph.input:
        print(f"  Name: {input.name}, Shape: {[dim.dim_param or dim.dim_value for dim in input.type.tensor_type.shape.dim]}")
    print(f"Outputs for {onnx_file}:")
    for output in onnx_model.graph.output:
        print(f"  Name: {output.name}, Shape: {[dim.dim_param or dim.dim_value for dim in input.type.tensor_type.shape.dim]}")
    print("-" * 50)

Checking /content/yolov9/best-b.onnx...
/content/yolov9/best-b.onnx is valid!
Inputs for /content/yolov9/best-b.onnx:
  Name: images, Shape: ['batch', 3, 'height', 'width']
Outputs for /content/yolov9/best-b.onnx:
  Name: output0, Shape: ['batch', 3, 'height', 'width']
  Name: 1877, Shape: ['batch', 3, 'height', 'width']
--------------------------------------------------
Checking /content/yolov9/last-b.onnx...
/content/yolov9/last-b.onnx is valid!
Inputs for /content/yolov9/last-b.onnx:
  Name: images, Shape: ['batch', 3, 'height', 'width']
Outputs for /content/yolov9/last-b.onnx:
  Name: output0, Shape: ['batch', 3, 'height', 'width']
  Name: 1877, Shape: ['batch', 3, 'height', 'width']
--------------------------------------------------
Checking /content/yolov9/best-s.onnx...
/content/yolov9/best-s.onnx is valid!
Inputs for /content/yolov9/best-s.onnx:
  Name: images, Shape: ['batch', 3, 'height', 'width']
Outputs for /content/yolov9/best-s.onnx:
  Name: output0, Shape: ['batch', 3, '

In [ ]:
 import onnx
 from onnxconverter_common import float16

In [ ]:
model = onnx.load("/content/last-s.onnx")

In [ ]:
model_fp16 = float16.convert_float_to_float16(model)

In [ ]:
 onnx.save(model_fp16, "last_S_fp16.onnx")

In [ ]:
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType
import numpy as np
import onnx

onnx_model_path = "onnx_path"
quantized_model_path = "output_path"

class YOLODataReader(CalibrationDataReader):
    def __init__(self):
        self.data = iter([{"images": np.random.rand(1, 3, 640, 640).astype(np.float32)}])

    def get_next(self):
        return next(self.data, None)

    def rewind(self):
        self.data = iter([{"images": np.random.rand(1, 3, 640, 640).astype(np.float32)}])

quantize_static(
    onnx_model_path,
    quantized_model_path,
    YOLODataReader(),
    weight_type=QuantType.QInt8,
    activation_type=QuantType.QUInt8
)

print(f"✅ Quantized model saved at {quantized_model_path}")


✅ Quantized model saved at last-s-int8.onnx
